In [1]:
import pandas as pd

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=e0f20004db48538ee8212b2cc59134733a04829723ce7b3224c90fb2e70b3fbb
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [4]:
bowl_df = pd.read_csv('AllSeasonPurpleCapData.csv')

In [6]:
bowl_df.head()

,season,fullName,TotalWickets,Overs,RunsConceded,AvgWickets,TotalMatches,MaxWickets,FoursConceded,SixesConceded,AvgEconomyRate,AvgRate,StrRate
0,2020,Krunal Pandya,6,50,380,0.375000,16,2,27,11,7.828125,63.333333,50.000000
1,2020,Marcus Stoinis,13,29,283,1.000000,13,3,23,14,9.193846,21.769231,13.384615
2,2016,Ravichandran Ashwin,10,44,319,0.714286,14,4,22,10,7.612857,31.900000,26.400000
3,2016,Mitchell McClenaghan,17,52,436,1.214286,14,4,47,16,8.202143,25.647059,18.352941
4,2016,Harbhajan Singh,9,49,392,0.642857,14,2,28,16,8.112857,43.555556,32.666667


In [7]:
X = bowl_df[['TotalMatches','Overs','RunsConceded','AvgWickets','AvgRate','AvgEconomyRate','StrRate']]
y = bowl_df[['TotalWickets']]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [9]:
bowl_model = LinearRegression()
bowl_model.fit(X_train, y_train)

LinearRegression()

In [11]:
y_pred = bowl_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error", mse)

Mean Squared Error 2.808570687887073


In [41]:
print(len(y_pred))

276


In [42]:
from sklearn.linear_model import LinearRegression
r2_score = bowl_model.score(X_test,y_test)
print(r2_score*100,'%')

92.16097958697345 %


In [18]:
from pyspark.sql.types import *

In [19]:
bowl23_schema = StructType([StructField('fullName', StringType(), True),
  StructField('Matches', IntegerType(), True),
  StructField('TotalMatches', IntegerType(), True),
  StructField('Overs', FloatType(), True),
  StructField('RunsConceded', IntegerType(), True),
  StructField('TotalWickets', IntegerType(), True),
  StructField('AvgRate', FloatType(), True),
  StructField('AvgEconomyRate', FloatType(), True),
  StructField('StrRate', FloatType(), True),
  StructField('4w', IntegerType(), True),
  StructField('5w', IntegerType(), True)])

In [22]:
from pyspark.sql import SparkSession

In [23]:
spark = SparkSession.builder.appName("IPL").getOrCreate()

In [25]:
curr_data = spark.read.option("header",True).schema(bowl23_schema).csv("2023purplecap.csv")

In [26]:
curr_data = curr_data.withColumn("AvgWickets", curr_data['TotalWickets']/curr_data['TotalMatches'])

In [27]:
curr_data.show()

+-------------------+-------+------------+-----+------------+------------+-------+--------------+-------+---+---+------------------+
|           fullName|Matches|TotalMatches|Overs|RunsConceded|TotalWickets|AvgRate|AvgEconomyRate|StrRate| 4w| 5w|        AvgWickets|
+-------------------+-------+------------+-----+------------+------------+-------+--------------+-------+---+---+------------------+
|   Tushar Deshpande|     11|          11| 38.2|         396|          19|  20.84|         10.33|   12.1|  0|  0|1.7272727272727273|
|     Mohammad Shami|     10|          10| 39.0|         274|          18|  15.22|          7.02|   13.0|  1|  0|               1.8|
|        Rashid Khan|     10|          10| 40.0|         322|          18|  17.88|          8.05|  13.33|  0|  0|               1.8|
|      Piyush Chawla|     10|          10| 39.0|         280|          17|  16.47|          7.17|  13.76|  0|  0|               1.7|
|     Arshdeep Singh|     10|          10| 36.5|         361|        

In [28]:
curr_data.write.option("header",True) \
        .mode("overwrite") \
        .csv("transformedpurplecapdata2023")

In [29]:
current_data = curr_data.toPandas()

In [30]:
X = current_data[['TotalMatches','Overs','RunsConceded','AvgWickets','AvgRate','AvgEconomyRate','StrRate']]
# y = bowldf[['TotalWickets']]

In [32]:
predicted_wickets = bowl_model.predict(X)

In [34]:
import numpy as np

In [35]:
ps = np.round(predicted_wickets)

In [58]:
for i in range(0,len(ps)):
  print(current_data['fullName'][i],ps[i])

Tushar Deshpande [14.]
Mohammad Shami [15.]
Rashid Khan [15.]
Piyush Chawla [15.]
Arshdeep Singh [13.]
Ravindra Jadeja [13.]
Mohammed Siraj [13.]
Varun Chakaravarthy [13.]
Ravichandran Ashwin [12.]
Yuzvendra Chahal [12.]
Ravi Bishnoi [11.]
Harshal Patel [11.]
Mayank Markande [11.]
Mark Wood [13.]
Noor Ahmad [10.]
Matheesha Pathirana [10.]
Trent Boult [10.]
Mitchell Marsh [8.]
Moeen Ali [8.]
Suyash Sharma [9.]
Nathan Ellis [9.]
Mohit Sharma [7.]
Kuldeep Yadav [9.]
Sandeep Sharma [9.]
Bhuvneshwar Kumar [9.]
Karn Sharma [8.]
T Natarajan [8.]
Marco Jansen [8.]
Jason Behrendorff [8.]
Naveen-ul-Haq [8.]
Axar Patel [8.]
Wanindu Hasaranga [7.]
Maheesh Theekshana [8.]
Sunil Narine [8.]
Alzai Joseph [7.]
Anrich Nortje [8.]
Sam Cuan [9.]
Riley Meredith [7.]
Andre Russell [6.]
Ishant Sharma [6.]
Krunal Pandya [6.]
Amit Mishra [6.]
Josh Little [7.]
Khaleel Ahmed [6.]
Avesh Khan [6.]
Wayne Parnell [7.]
Vyshak Vijaykumar [6.]
Mukesh Kumar [7.]
Marcus Stoinis [5.]
Yash Thakur [5.]
Kagiso Rabada [6.]
A

Kartik Tyagi [2.]


In [36]:
purple_cap_winner_index = ps.argmax()

In [37]:
pcw = current_data.iloc[purple_cap_winner_index]

In [43]:
print(current_data['fullName'][purple_cap_winner_index])


Mohammad Shami
